# Background & motivation

Big data is a general term referring to several challenges encountered when dealing with a large amount of data. These challenges are known as **the 5 V's** - Volume, Variety, Velocity, Variability and Veracity. Traditionally, the 5 V's were solved by the **scale-up** approach, which means upgrading the resources - better servers, faster connections, larger memories, etc. This is now replaced by the **scale-out** approach, which is based on **parallelizing** tasks over a **cluster** of "weak" servers.

It is important to notice that the 5 V's are **infrastructural** challenges and are not necessarily related to any business or analytical problem. This is important to notice, because the confusion between the two is very common, making people think that data science and big data are similar expertise. 

Let's discuss some important terms:

* [**Cluster**][cluster] - a set of loosely or tightly connected computers that work together so that, in many respects, they can be viewed as a single system. Any cluster has a **master** (or main) and **nodes** (or slaves), which offer two main advantages - **Fault-tolerance** and **Data locality**. 
* [**MapReduce**][mapreduce] - a programming model and an associated implementation for processing and generating big data sets with a parallel, distributed algorithm on a cluster.
* [**Hadoop**][hadoop] - an open-source software framework used for distributed storage and processing of big data sets using the MapReduce programming model.
    * **Hadoop Data File System (HDFS)** - a distributed, scalable, and portable file system for the Hadoop framework.
    * **Hadoop ecosystem** - collection of additional software packages that can be installed on top of or alongside Hadoop (e.g. Yarn, Pig, Hive, Sqoop, Mahout, etc.). Many of the big data tools aim to **abstract** the parallelization.
    * **Hadoop distribution** - companies providing Hadoop-based software, support, services and training (e.g. Cloudera, Hortonworks, etc.)

[cluster]: https://en.wikipedia.org/wiki/Computer_cluster "Computer cluster - Wikipedia"
[mapreduce]: https://en.wikipedia.org/wiki/MapReduce "MapReduce - Wikipedia"
[hadoop]: https://en.wikipedia.org/wiki/Apache_Hadoop "Hadoop - Wikipedia"
[hdfs]: https://en.wikipedia.org/wiki/Apache_Hadoop#HDFS "HDFS - Wikipedia"

# Spark

## Basics

[Spark](https://spark.apache.org/docs/latest/) is a framework for programming with an abstraction of the map-reduce paradigm. Its main data structure (**RDD**) allows better utilization of the memory of the nodes, and this made it very popular in recent years. Spark was originally part of the Hadoop ecosystem, however it was so useful, that eventually it was decided to make it available as a stand-alone framework. 

Spark is made of 5 building blocks:

* Spark core - the fundamentals components of the language. It provides distributed task dispatching, scheduling, and basic I/O functionalities, exposed through an API centered on the RDD abstraction.
* Spark SQL - tools for working with DataFrames. It provides an API for embedding SQL scripts, as well as connections with an ODBC/JDBC server.
* Spark streaming - facilitates tasks witha a data stream. It ingests data in mini-batches and performs RDD transformations on those mini-batches of data.
* Spark MLlib - distributed versions of various machine learning (ML) algorithms.
* Spark GraphX - graph processing framework.

[PySpark](https://spark.apache.org/docs/latest/api/python/index.html) is the official Python API for Spark.

## Working with Spark

The first thing a Spark program must do is to create a `SparkContext` object (traditionally symbolized as the variable `sc`), which tells Spark how to access a cluster. (To create a `SparkContext` you might first need to build a `SparkConf` object that contains information about your application.).

Spark is written in [Scala](https://en.wikipedia.org/wiki/Scala_(programming_language)), but it suports APIs for Java, R and Python. We will work with the official Python API - [**PySpark**](https://spark.apache.org/docs/latest/api/python/index.html).

In [0]:
# from pyspark import SparkContext
# sc = SparkContext.getOrCreate()

In [0]:
sc

# RDDs

[**Resilient Distributed Dataset (RDD)**](https://spark.apache.org/docs/latest/rdd-programming-guide.html) is the main data object in Spark and it is an abstraction of the data parallelization. This means that we can work with a single RDD, where in fact its data, as well as its processing, may be distributed in the cluster.

Data sharing is slow in MapReduce due to replication, serialization, and disk IO (Actually, most Hadoop applications spend more than 90% of the time doing HDFS read-write operations.). Recognizing this problem, RDDs support **in-memory** processing computation. This means, it stores the state of memory as an object across the jobs and the object is sharable between those jobs.

> **Notes:**
> * RDDs are immutable, which has a great influence on the appearence of Spark code.
> * If the elements of an RDD are tuples (which is a Spark data type equivalent to Python tuples of length 2), then each tuple is automatically recognized as a pair of a **key** and a **value**, and we say that it is a *pair RDD*.

## Playground

`SparkContext` has various methods for creating RDDs from various sources, especially within Hadoop, e.g. [newAPIHadoopFile()](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.SparkContext.newAPIHadoopFile), however these are less common due to specific loaders we will see later. For this notebook we will suffice with two RDD creators - `parallelize()` and `textFile()`.

### `parallelize()`

The [`parallelize()`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.parallelize) method takes a *Pythonic* iterable object and creates an RDD with its elements.

In [0]:
my_list = [12, 23, 34, 45, 56, 67, 78, 89, 90]
rdd1 = sc.parallelize(my_list)
print(type(rdd1))
# print(rdd1)

`rdd1` is of an RDD type, and we can look in the [`RDD` API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.html#rdd-apis) for all its available methods. See some self-explanatory illustrations below:

In [0]:
rdd2 = rdd1.map(lambda x: x/2)
rdd2.collect()

In [0]:
rdd3 = rdd1.filter(lambda x: x%2==0)
rdd3.collect()

In [0]:
rdd4 = rdd1.groupBy(lambda x: x%3)
print(list(rdd4.collect()))
print(list(rdd4.collect()[1][1]))

In [0]:
rdd4.mapValues(lambda iterable: list(iterable)).collect()

In [0]:
rdd4.map(lambda tup: list(tup[1])).collect()

> **Note:** Defining RDDs with iterators returns a `PipelineRDD` object, which doesn't support the standard RDD methods. Compare with `sc.parallelize(range(100))`.

### `textFile()`

Alternatively, we can create an RDD from a text file using the [`textFile()`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.textFile) method. We demonstrate with the text of the book *Moby Dick* (available [here](https://drive.google.com/drive/folders/1NuT5ic0FHCXkoQoQnkBE5e1KVknm8SXP?usp=sharing))

In [0]:
text = sc\
    .textFile("dbfs:/FileStore/texts/melville_moby_dick.txt")

In [0]:
type(text)

Like with normal text files, the RDD is made of the lines in the book, which were separated in the file with `\n`.

In [0]:
text.take(15)

## Transformations and actions

RDD **transformations** are operations applied on RDDs to yield a new RDD. On the other hand, **actions** are operations applied on RDDs to yield a non-RDD result (number, string, list, etc.). 

Here are some examples:

* Transformations:
    * _map(func)_ - Returns a new distributed dataset, formed by passing each element of the source through a function func.
    * _filter(func)_ - Returns a new dataset formed by selecting those elements of the source on which func returns true.
    * _groupByKey()_ - When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable(V)) pairs.
    * _sortByKey([ascending])_ - When called on a dataset of (K, V) pairs where K implements Ordered, returns a dataset of (K, V) pairs sorted by keys in ascending or descending order, as specified in the Boolean ascending argument.
* Actions:
    * _count()_ - Returns the number of elements in the dataset.
    * _take(n)_ - Returns an array with the first n elements of the dataset. 
    * _saveAsTextFile(path)_ - Writes the elements of the dataset as a text file (or set of text files) in a given directory in the local filesystem, HDFS or any other Hadoop-supported file system. Spark calls _toString()_ on each element to convert it to a line of text in the file.

> **Warning:** There is an action called [`collect()`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) which is similar to the _take()_ action, but returns **all** the elements of the RDD. This action collects the relevant elements to the master of the cluster, and can easily crush the system. This is why it is usually useful after a filter or other operation that returns a sufficiently small subset of the data.

> **Note:** In most cases one applies a chain of transformations which ends with an action. Each RDD in such dependency chain has a pointer (dependency) to its parent RDD. Spark is **lazy**, so nothing will be executed until an action will trigger the chain. Therefore, RDD transformation is not a set of data but is a step in a program (might be the only step) telling Spark how to get data and what to do with it.
>
> **Reference:** [Explanantion about lazy calculation by DataFlair](https://data-flair.training/blogs/apache-spark-lazy-evaluation/)

# Spark SQL & DataFrames

Spark SQL is a Spark module for structured data processing. Unlike the basic Spark RDD API, the interfaces provided by Spark SQL provide Spark with more information about the structure of both the data and the computation being performed. Internally, Spark SQL uses this extra information to perform extra optimizations. There are several ways to interact with Spark SQL including plain SQL and the **Dataframe** API. When computing a result the same execution engine is used, independent of which API/language you are using to express the computation.

## Playground

A `SparkContext` is what you need for working with the Spark core elements, however the entry point into Spark SQL functionality is the `SparkSession` object (traditionally symbolized as the variable `spark`).

In [0]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.getOrCreate()

In [0]:
spark

`spark` has many useful types of [`DataFrameReader`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader), which upload *structured* data from various sources (e.g. relational databases, Kafka topics, etc.) into Spark DataFrames. In this notebook we will focus on the most simple reader - CSV reader.

In [0]:
dessert = spark.read.csv("/FileStore/tables/dessert.csv", 
                         header=True, 
                         inferSchema=True)

In [0]:
type(dessert)

> **Note:** We use the default `inferSchema=True`, but you can also specify your own schema when reading the data (or before). To do that you need to use [`StructType`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.types.StructType), as explained in [this introduction](https://mungingdata.com/apache-spark/dataframe-schema-structfield-structtype/) by Munging Data, and will be coverred later in the course.

In [0]:
dessert.show(5)

In [0]:
display(dessert)

Each record in the "dessert" dataset describes a group visit at a restaurant. We demonstrate below some self-explanatory DataFrame manipulations (see [DataFrame API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.html) for more details).

In [0]:
dessert.printSchema()

In [0]:
dessert = dessert\
    .withColumnRenamed('dessert', 'purchase')\
    .withColumnRenamed('day.of.week', 'weekday')\
    .withColumnRenamed('num.of.guests', 'guests')
dessert.show(5)

In [0]:
dessert = dessert\
    .withColumn('no_purchase', ~dessert.purchase)
dessert.show(5)

In [0]:
dessert = dessert\
    .withColumn("purchase", dessert["purchase"].cast("integer"))\
    .withColumn("no_purchase", dessert["no_purchase"].cast("integer"))
dessert.show(5)

In [0]:
import pyspark.sql.functions as f

In [0]:
buyers = dessert\
    .groupBy('guests')\
    .agg(f.sum(dessert.purchase).alias('buyers'), 
         f.sum(dessert.no_purchase).alias('non_buyers'))
buyers.show()

In [0]:
buyers.toPandas().set_index('guests').sort_index().plot.bar()

## Transformation and actions

Dataframe is a special type of RDD, and as such it supports all RDD operations (as an RDD of `Row` elements) and an additional large set of unique attributes and methods:

* Attributes
    * `column` - Returns all column names as a list.
    * `rdd` - Returns the content as an RDD (of `Row` elements).
    * `schema` - Returns the schema of this DataFrame (as a `StructType`).
* Methods
    * `crosstab(col1, col2)` - Computes a pair-wise frequency table of the given columns (pivot table).
    * `drop(*cols)` - Returns a new DataFrame that drops the specified column.
    * `head(n=None)` - Returns the first n rows.
    * `orderBy(*cols)` - Returns a new DataFrame sorted by the specified column(s).
    * `printSchema()` - Prints out the schema in a tree format.
    * `where(condition)` - an alias for `filter()`